In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
from fiona.crs import from_epsg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold,GroupKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import xlsxwriter
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model
from sklearn import metrics
import datetime
from scipy.stats import gaussian_kde
from sklearn import ensemble
from rasterio.plot import show
import glob
import os
import rasterio
from rasterio import merge
import matplotlib.image as mpimg
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold,GroupKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import xlsxwriter
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model
from sklearn import metrics
import datetime
from scipy.stats import gaussian_kde
from sklearn import ensemble
from rasterio.plot import show
import glob
import os
import rasterio
from rasterio import merge
import matplotlib.image as mpimg
from sklearn import linear_model
from sklearn.svm import SVR
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from shapely.geometry import Polygon

In [8]:
# --- imports ---
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn import metrics
import xgboost as xgb

# --- data ---
workbook = 'field_point.xlsx'
sheet = 'all'
df = pd.read_excel(workbook, sheet)

# features/target
X = df.iloc[:, 5:-1].values   # all cols except last (from col 5)
Y = df.iloc[:, -1].values     # last col as target

# ---------- ONE GRID ONLY (≈500 m) ----------
cell_size = 0.005  # degrees (~500 m). This is the FIRST value in your old loop.

# points -> GeoDataFrame (WGS84)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# save & reload (kept your behavior)
gdf.to_file("SA.shp")
SA = gpd.read_file("SA.shp")

# build the grid (in degrees, like your original)
minx, miny, maxx, maxy = gdf.total_bounds
rows = int((maxy - miny) // cell_size + 3)
cols = int((maxx - minx) // cell_size + 3)

polygons = []
for i in range(rows):
    for j in range(cols):
        xmin = minx - 1.5 * cell_size + j * cell_size
        xmax = xmin + cell_size
        ymin = miny - 1.5 * cell_size + i * cell_size
        ymax = ymin + cell_size
        polygons.append(Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]))

grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
# add a real FID column so later lookups work
grid_gdf['FID'] = np.arange(len(grid_gdf), dtype=int)

grid_shapefile = f"grid_{cell_size}.shp"
grid_gdf.to_file(grid_shapefile)

# load grid
grid = gpd.read_file(grid_shapefile)

# assign each point to a grid cell (within -> nearest fallback)
grid_df = pd.DataFrame(columns=[f'grid_{cell_size}'])
for idx, point in SA.iterrows():
    containing = grid[grid.contains(point.geometry)]
    if not containing.empty:
        field_value = int(containing.iloc[0]['FID'])
        grid_df.loc[idx] = [field_value]
    else:
        nearest = grid.loc[grid.distance(point.geometry).idxmin()]
        field_value = int(nearest['FID'])
        grid_df.loc[idx] = [field_value]

# GroupKFold on that single grid
block = grid_df[f'grid_{cell_size}'].astype(int).values
group_kfold = GroupKFold(n_splits=3)
block_kfold = group_kfold.split(X, Y, block)
train_indices, test_indices = [list(traintest) for traintest in zip(*block_kfold)]
block_cv = [*zip(train_indices, test_indices)]

# model + CV prediction
model = xgb.XGBRegressor()
Y_pred = cross_val_predict(model, X, Y, cv=block_cv)

# metrics
r2 = metrics.r2_score(Y, Y_pred)
mae = float(metrics.mean_absolute_error(Y, Y_pred))
rmse = float(np.sqrt(metrics.mean_squared_error(Y, Y_pred)))

print("Single grid run (cell_size = 0.005° ≈ 500 m)")
print(f"R²   : {r2:.4f}")
print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")


Single grid run (cell_size = 0.005° ≈ 500 m)
R²   : 0.3068
MAE  : 816.94
RMSE : 1085.44


In [9]:

workbook = 'field_point.xlsx'
sheet = 'all'
df = pd.read_excel(workbook, sheet)

# features/target
X = df.iloc[:, 5:-1].values   # all cols except last (from col 5)
Y = df.iloc[:, -1].values     # last col as target

# ---------- ONE GRID ONLY (≈500 m) ----------
cell_size = 0.005  # degrees (~500 m). This is the first value in your old loop.

# points -> GeoDataFrame (WGS84)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# save & reload (kept your behavior)
gdf.to_file("SA.shp")
SA = gpd.read_file("SA.shp")

# build the grid (in degrees, like your original)
minx, miny, maxx, maxy = gdf.total_bounds
rows = int((maxy - miny) // cell_size + 3)
cols = int((maxx - minx) // cell_size + 3)

polygons = []
for i in range(rows):
    for j in range(cols):
        xmin = minx - 1.5 * cell_size + j * cell_size
        xmax = xmin + cell_size
        ymin = miny - 1.5 * cell_size + i * cell_size
        ymax = ymin + cell_size
        polygons.append(Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]))

grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
# add a real FID column so later lookups work
grid_gdf['FID'] = np.arange(len(grid_gdf), dtype=int)

grid_shapefile = f"grid_{cell_size}.shp"
grid_gdf.to_file(grid_shapefile)

# load grid
grid = gpd.read_file(grid_shapefile)

# assign each point to a grid cell (within -> nearest fallback)
grid_df = pd.DataFrame(columns=[f'grid_{cell_size}'])
for idx, point in SA.iterrows():
    containing = grid[grid.contains(point.geometry)]
    if not containing.empty:
        field_value = int(containing.iloc[0]['FID'])
        grid_df.loc[idx] = [field_value]
    else:
        # nearest fallback
        nearest = grid.loc[grid.distance(point.geometry).idxmin()]
        field_value = int(nearest['FID'])
        grid_df.loc[idx] = [field_value]

# GroupKFold on that single grid
block = grid_df[f'grid_{cell_size}'].astype(int).values
group_kfold = GroupKFold(n_splits=3)
block_kfold = list(group_kfold.split(X, Y, block))

# ---- Train per fold and average metrics ----
r2s, maes, rmses = [], [], []
for train_idx, test_idx in block_kfold:
    model = xgb.XGBRegressor()
    model.fit(X[train_idx], Y[train_idx])
    y_pred = model.predict(X[test_idx])
    y_true = Y[test_idx]

    r2s.append(metrics.r2_score(y_true, y_pred))
    maes.append(metrics.mean_absolute_error(y_true, y_pred))
    rmses.append(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

# mean scores across the 3 folds
r2_mean = float(np.mean(r2s))
mae_mean = float(np.mean(maes))
rmse_mean = float(np.mean(rmses))

print("Single grid run (cell_size = 0.005° ≈ 500 m) — mean of 3 folds")
print(f"R²   : {r2_mean:.4f}")
print(f"MAE  : {mae_mean:.2f}")
print(f"RMSE : {rmse_mean:.2f}")


Single grid run (cell_size = 0.005° ≈ 500 m) — mean of 3 folds
R²   : 0.3016
MAE  : 816.94
RMSE : 1084.79


In [11]:
from itertools import product


In [12]:

workbook = 'field_point.xlsx'
sheet = 'all'
df = pd.read_excel(workbook, sheet)

# features/target
X = df.iloc[:, 5:-1].values   # all cols except last (from col 5)
Y = df.iloc[:, -1].values     # last col as target

# ---------- ONE GRID ONLY (≈500 m) ----------
cell_size = 0.005  # degrees (~500 m). This is the first value in your old loop.

# points -> GeoDataFrame (WGS84)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# save & reload (kept your behavior)
gdf.to_file("SA.shp")
SA = gpd.read_file("SA.shp")

# build the grid (in degrees, like your original)
minx, miny, maxx, maxy = gdf.total_bounds
rows = int((maxy - miny) // cell_size + 3)
cols = int((maxx - minx) // cell_size + 3)

polygons = []
for i in range(rows):
    for j in range(cols):
        xmin = minx - 1.5 * cell_size + j * cell_size
        xmax = xmin + cell_size
        ymin = miny - 1.5 * cell_size + i * cell_size
        ymax = ymin + cell_size
        polygons.append(Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]))

grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
# add a real FID column so later lookups work
grid_gdf['FID'] = np.arange(len(grid_gdf), dtype=int)

grid_shapefile = f"grid_{cell_size}.shp"
grid_gdf.to_file(grid_shapefile)

# load grid
grid = gpd.read_file(grid_shapefile)

# assign each point to a grid cell (within -> nearest fallback)
grid_df = pd.DataFrame(columns=[f'grid_{cell_size}'])
for idx, point in SA.iterrows():
    containing = grid[grid.contains(point.geometry)]
    if not containing.empty:
        field_value = int(containing.iloc[0]['FID'])
        grid_df.loc[idx] = [field_value]
    else:
        # nearest fallback (distance in degrees; matches your original approach)
        nearest = grid.loc[grid.distance(point.geometry).idxmin()]
        field_value = int(nearest['FID'])
        grid_df.loc[idx] = [field_value]

# GroupKFold on that single grid
block = grid_df[f'grid_{cell_size}'].astype(int).values
n_groups = pd.Series(block).nunique()
if n_groups < 2:
    raise ValueError(f"Not enough unique grid blocks for CV (found {n_groups}).")
n_splits = 3 if n_groups >= 3 else 2
group_kfold = GroupKFold(n_splits=n_splits)
block_kfold = list(group_kfold.split(X, Y, block))

# ---------- Hyperparameter grid (Zhang et al., 2022 ranges) ----------
max_depth_grid      = list(range(1, 6))              # 1..5
n_estimators_grid   = list(range(100, 1001, 100))    # 100..1000 step 100
learning_rate_grid  = [0.1, 0.2, 0.3]               # 0.1..0.3 step 0.1
colsample_grid      = [i/10 for i in range(1, 10)]  # 0.1..0.9 step 0.1

# ---------- Grid search loop ----------
results = []
for md, ne, lr, cs in product(max_depth_grid, n_estimators_grid, learning_rate_grid, colsample_grid):
    r2s, maes, rmses = [], [], []
    for train_idx, test_idx in block_kfold:
        model = xgb.XGBRegressor(
            max_depth=md,
            n_estimators=ne,
            learning_rate=lr,
            colsample_bytree=cs,
            random_state=42,  # for repeatability
            n_jobs=1          # stable; change to -1 if you want parallel
        )
        model.fit(X[train_idx], Y[train_idx])
        y_pred = model.predict(X[test_idx])
        y_true = Y[test_idx]

        r2s.append(metrics.r2_score(y_true, y_pred))
        maes.append(metrics.mean_absolute_error(y_true, y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

    results.append({
        'max_depth': md,
        'n_estimators': ne,
        'learning_rate': lr,
        'colsample_bytree': cs,
        'R2_mean': float(np.mean(r2s)),
        'MAE_mean': float(np.mean(maes)),
        'RMSE_mean': float(np.mean(rmses))
    })

# ---------- Select the best combo by highest mean R² (tie-breakers: lowest RMSE, then lowest MAE) ----------
best = max(
    results,
    key=lambda d: (d['R2_mean'], -d['RMSE_mean'], -d['MAE_mean'])
)

# ---------- Print only the best ----------
print("Best hyperparameters (by mean R², with RMSE/MAE tie-breakers):")
print(f"  max_depth       = {best['max_depth']}")
print(f"  n_estimators    = {best['n_estimators']}")
print(f"  learning_rate   = {best['learning_rate']}")
print(f"  colsample_bytree= {best['colsample_bytree']}")
print("\nMean CV scores (3 folds):")
print(f"  R²   : {best['R2_mean']:.4f}")
print(f"  RMSE : {best['RMSE_mean']:.2f}")
print(f"  MAE  : {best['MAE_mean']:.2f}")


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn.model_selection import GroupKFold
from sklearn import metrics
import xgboost as xgb
from itertools import product


workbook = 'field_point.xlsx'
sheet = 'all'
df = pd.read_excel(workbook, sheet)

X = df.iloc[:, 5:-1].values  
Y = df.iloc[:, -1].values   


cell_size = 0.005  


geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")


gdf.to_file("SA.shp")
SA = gpd.read_file("SA.shp")


minx, miny, maxx, maxy = gdf.total_bounds
rows = int((maxy - miny) // cell_size + 3)
cols = int((maxx - minx) // cell_size + 3)

polygons = []
for i in range(rows):
    for j in range(cols):
        xmin = minx - 1.5 * cell_size + j * cell_size
        xmax = xmin + cell_size
        ymin = miny - 1.5 * cell_size + i * cell_size
        ymax = ymin + cell_size
        polygons.append(Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]))

grid_gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
# add a real FID column so later lookups work
grid_gdf['FID'] = np.arange(len(grid_gdf), dtype=int)

grid_shapefile = f"grid_{cell_size}.shp"
grid_gdf.to_file(grid_shapefile)

# load grid
grid = gpd.read_file(grid_shapefile)

# assign each point to a grid cell (within -> nearest fallback)
grid_df = pd.DataFrame(columns=[f'grid_{cell_size}'])
for idx, point in SA.iterrows():
    containing = grid[grid.contains(point.geometry)]
    if not containing.empty:
        field_value = int(containing.iloc[0]['FID'])
        grid_df.loc[idx] = [field_value]
    else:

        nearest = grid.loc[grid.distance(point.geometry).idxmin()]
        field_value = int(nearest['FID'])
        grid_df.loc[idx] = [field_value]


block = grid_df[f'grid_{cell_size}'].astype(int).values
n_groups = pd.Series(block).nunique()
if n_groups < 2:
    raise ValueError(f"Not enough unique grid blocks for CV (found {n_groups}).")
n_splits = 3 if n_groups >= 3 else 2
group_kfold = GroupKFold(n_splits=n_splits)
block_kfold = list(group_kfold.split(X, Y, block))


max_depth_grid      = list(range(1, 6))              
n_estimators_grid   = list(range(100, 1001, 100))    
learning_rate_grid  = [0.1, 0.2, 0.3]               
colsample_grid      = [i/10 for i in range(1, 10)] 


results = []
for md, ne, lr, cs in product(max_depth_grid, n_estimators_grid, learning_rate_grid, colsample_grid):
    r2s, maes, rmses = [], [], []
    for train_idx, test_idx in block_kfold:
        model = xgb.XGBRegressor(
            max_depth=md,
            n_estimators=ne,
            learning_rate=lr,
            colsample_bytree=cs,
            random_state=42,  
            n_jobs=1          
        )
        model.fit(X[train_idx], Y[train_idx])
        y_pred = model.predict(X[test_idx])
        y_true = Y[test_idx]

        r2s.append(metrics.r2_score(y_true, y_pred))
        maes.append(metrics.mean_absolute_error(y_true, y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

    results.append({
        'max_depth': md,
        'n_estimators': ne,
        'learning_rate': lr,
        'colsample_bytree': cs,
        'R2_mean': float(np.mean(r2s)),
        'MAE_mean': float(np.mean(maes)),
        'RMSE_mean': float(np.mean(rmses))
    })


best = max(
    results,
    key=lambda d: (d['R2_mean'], -d['RMSE_mean'], -d['MAE_mean'])
)

print("Best hyperparameters (by mean R², with RMSE/MAE tie-breakers):")
print(f"  max_depth       = {best['max_depth']}")
print(f"  n_estimators    = {best['n_estimators']}")
print(f"  learning_rate   = {best['learning_rate']}")
print(f"  colsample_bytree= {best['colsample_bytree']}")
print("\nMean CV scores (3 folds):")
print(f"  R²   : {best['R2_mean']:.4f}")
print(f"  RMSE : {best['RMSE_mean']:.2f}")
print(f"  MAE  : {best['MAE_mean']:.2f}")
